# 부스팅의 개념

- 부스팅(boosting)
  - 학습 라운드를 차례로 진행하면서 각 예측이 틀린 데이터에 점점 가중치를 주는 방식
  - 라운드별로 잘못 분류된 데이터를 좀 더 잘 분류하는 모델로 만들어 최종적으로 모델들의 앙상블을 만드는 방식
    - 배깅 알고리즘이 처음 성능을 측정하기 위한 기준(baseline) 알고리즘으로 많이 사용된다면, 부스팅 알고리즘은 높은 성능을 내야 하는 상황에서 가장 좋은 선택지


- 부스팅(boosting)
  - 첫 번째 라운드 결과 모델 (1)에서 A점은 오차가 큰 부분
  - 두 번째 라운드에서 오답으로 분류된 A에 가중치를 줘 학습
  - 다시 오류가 큰 B 영역에 가중치를 둬 모델 (3) 개발
  - 틀린 부분만 집중해서 모델들을 순차적으로 만들고, 해당 모델들은 최종적으로 앙상블


# 배깅과 부스팅의 차이점

- 병렬화 가능 여부
  - 배깅은 데이터가 n개바라면 n개의 CPU로 한번에 처리하도록 구조를 설계할 수 있음
  - 배깅은 데이터를 나눠 데이터마다 조금씩 다른 모델을 생성
  - 부스팅은 단계적으로 모델들을 생성하고 해당 모델들의 성능을 측정한 후, 다음 단계로 넘어가 병렬화를 지원하지 않음
  - 부스팅은 배깅에 비해 속도가 매우 떨어짐
- 기준 추정치
  - 배깅 개별 모델들은 높은 과대적합으로 모델의 분산이 높음
  - 부스팅은 각각의 모델에 편향이 높은 기준 추정치를 사용하여 개별 모델들은 과소적합이 발생하지만 전체적으로 높은 성능을 낼 수 있는 방향으로 학습
  - 부스팅 모델의 이러한 특징을 약한 학습자(weak learner)라고 부름
- 성능 차이
  - 부스팅은 기본적으로 비용이 높은 알고리즘
    - 비용: 속도나 시간
  - 배깅은 데이터의 부분집합에 대해 학습을 수행하기 때문에 부스팅보다 좋은 성능을 내기는 어려움
  - 초기 성능을 측정할 때는 배깅, 이후의 성능 측정은 부스팅으로 하는 것이 가장 일반적인 접근


# 에이다부스트

- 에이다부스트
  - 부스팅 알고리즘 중 대표적인 알고리즘
  - 에이다부스트(AdaBoost)
    - 매 라운드마다 인스턴스, 즉 개발 데이터의 가중치를 계산하는 방식
    - 매 라운드마다 틀린 값이 존재하고 해당 인스턴스에 가중치를 추가로 주어 가중치를 기준으로 재샘플린(resampling)
- 알고리즘
  - 모든 샘플의 가중치 값을 데이터 개수를 기준으로 1/N로 초기화한 다음
  - 데이터의 가중치를 사용해 모델을 학습시킨다
  - 해당 분류기의 오류를 계산한다
  - 해당 분류기의 가중치를 생성한다
  - 모델의 가중치를 사용하여 각 데이터의 가중치를 업데이트
  - 최종 결과물은 각 모델 가중치와 모델 결과값의 가중합을 연산하여 계산


# 에이다부스트와 스텀프

- 에이다부스트와 스텀프
  - 스텀프(stump)는 그루터기라는 뜻으로 나무의 윗부분을 자르고 아랫부분만 남은 상태
  - 에이다부스트에서 스텀프는 학습할 때 큰 나무를 사용하여 학습하는 것이 아닌 나무의 그루터기만을 사용하여 학습한다는 개념
  - 1뎁스 또는 2뎁스 정도의 매우 간단한 모델을 여러 개 만들어 학습한 후, 해당 모델들의 성능을 에이다부스트 알고리즘을 적용하여 학습하는 형태


# 에이다부스트 실습

- 사이킷런으로 에이다부스트 실습하기
  - 사이킷런에서 제공하는 AdaBoostClassifier를 사용


In [1]:
import numpy as np

X = np.load("./titanic/titanic_X_train.npy")
y = np.load("./titanic/titanic_y_train.npy")

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

eclf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=500)

In [9]:
from sklearn.model_selection import cross_val_score

cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8279248397130706)

- 비교군으로 RandomForestClassifier를 생성


In [10]:
from sklearn.ensemble import RandomForestClassifier

eclf = RandomForestClassifier(n_estimators=500)
cross_val_score(eclf, X, y, cv=5).mean()

np.float64(0.8031866945978544)

- GridSearchCV를 사용하여 가장 좋은 모델을 찾아 모델의 성능을 향상


In [11]:
eclf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=500)

params = {
    "estimator__criterion": [
        "gini",
        "entropy",
    ],  # 결정트리의 분할기준: 지니 불순도, 정보 이득 기준
    "estimator__max_features": [
        7,
        8,
    ],  # 측징의 최대 수
    "estimator__max_depth": [1, 2],  # 최대 깊이
    "n_estimators": [23, 24, 25, 26, 27],  # 추정기의 수
    "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6],  # 학습률
}


In [12]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X, y)

In [13]:
grid.best_score_

np.float64(0.8312892782327175)

In [14]:
grid.best_params_

{'estimator__criterion': 'gini',
 'estimator__max_depth': 2,
 'estimator__max_features': 7,
 'learning_rate': 0.6,
 'n_estimators': 24}

- feature*importances* 로 각 피쳐(feature)들이 모델에 영향을 미치는 정도를 나타낼 수 있음


In [15]:
grid.best_estimator_.feature_importances_

array([0.07112579, 0.07512415, 0.09225994, 0.10512874, 0.04493038,
       0.12002284, 0.01951929, 0.00344156, 0.        , 0.01642529,
       0.        , 0.        , 0.        , 0.0647143 , 0.00552223,
       0.16322963, 0.02397119, 0.04679664, 0.10738074, 0.        ,
       0.        , 0.        , 0.02886569, 0.00463882, 0.        ,
       0.00690277, 0.        ])